# Import Library

In [17]:
import pandas as pd
import numpy as np
import os
from datetime import datetime



# Merge Data

In [18]:
# Specify the directory where the CSV files are located
input_directory = r'D:\Project\Project_cycling\data\raw'

# Combine all CSV files in the directory into one DataFrame
dfs = []

for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(input_directory, filename)
        df = pd.read_csv(file_path)
        dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

# Obtaining the dimensions of a DataFrame

In [19]:
df.shape

(5667717, 13)

# Display the first 5 rows

In [20]:
df.head(5)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C2F7DD78E82EC875,electric_bike,2022-01-13 11:59:47,2022-01-13 12:02:44,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012800,-87.665906,42.012560,-87.674367,casual
1,A6CF8980A652D272,electric_bike,2022-01-10 08:41:56,2022-01-10 08:46:17,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012763,-87.665967,42.012560,-87.674367,casual
2,BD0F91DFF741C66D,classic_bike,2022-01-25 04:53:40,2022-01-25 04:58:01,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,41.925602,-87.653708,41.925330,-87.665800,member
3,CBB80ED419105406,classic_bike,2022-01-04 00:18:04,2022-01-04 00:33:00,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,41.983593,-87.669154,41.961507,-87.671387,casual
4,DDC963BFDDA51EEA,classic_bike,2022-01-20 01:31:10,2022-01-20 01:37:12,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,41.877850,-87.624080,41.884621,-87.627834,member


# Retrieve the column labels

In [21]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

# Dropping Irrelevant Columns

In [22]:
df=df[['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'end_station_name', 'member_casual']].copy()

# Retrieve the data types of the columns

In [23]:
df.dtypes

ride_id             object
rideable_type       object
started_at          object
ended_at            object
end_station_name    object
member_casual       object
dtype: object

# Convert the columns in a Pandas DataFrame to datetime format

In [24]:
df['started_at']=pd.to_datetime(df['started_at'])
df['ended_at']=pd.to_datetime(df['ended_at'])

# Count the number of duplicate rows

In [25]:
df.duplicated().sum()

0

# Calculate the count of missing values for each column

In [26]:
df.isna().sum()

ride_id                  0
rideable_type            0
started_at               0
ended_at                 0
end_station_name    892742
member_casual            0
dtype: int64

#  Remove rows that contain at least one missing (NaN) value.

In [27]:
df.dropna(inplace=True)

# Calculating the ride length in minutes

In [ ]:
# calculating the ride length in minutes
df['ride_length']=(df['ended_at']-df['started_at']).apply(lambda x: round(x.total_seconds()/60))
# filters out rows where the ride length is greater than 0 (rides with a positive duration)
df = df[df['ride_length'] > 0].copy()

# Extract date

In [ ]:
df['date'] = df['started_at'].apply(lambda x: x.date())

# Extract day of the week

In [ ]:

df['day_of_week'] = df['date'].apply(lambda x: x.strftime("%A"))

# Dropping Irrelevant Columns

In [ ]:
df=df[['ride_id', 'rideable_type', 'date','ride_length', 
       'day_of_week', 'end_station_name', 'member_casual']].copy()

# Writing the DataFrame to a CSV file

In [ ]:
df.to_csv(r'D:\Project\Project_cycling\data\processed\cyclisticfinal.csv', index=False)